In [10]:
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
all_data=dict()
# 從event id 為 0 一直試到event id 為30,000
for i in tqdm(range(30001)):
    res=requests.get('http://www.eventid.net/display.asp?eventid='+str(i)+'&source=')
    soup=bs(res.text, "lxml")
    table=soup.select('#box-table-doc')
    type_li=[]
    source_li=[]
    description_li=[]
# 開始剖析回傳的html code
# 每一筆資料都由一個"tr" tag所包覆
    for ele in table[0].select('tr'):
#      如果該筆文字由"type"開頭，則代表的是該筆log的type，而「type:」共5個字元，所以從第6個字元開始擷取至該行結束
        if ele.text.startswith('Type:'):
            type_li.append(ele.text[6:].strip())
#      如果該筆文字由"Source:"開頭，則代表的是該筆log的來源，而「Source:」共7字字，所以從第8個字元開始拮取至該行結束
        elif ele.text.startswith('Source:'):
            source_li.append(ele.text[8:].strip())
#      如果該筆文字由"Description:"開頭，則代表的是該筆log的描述，而「Description:」共11字字，所以從第8個字元開始拮取至該行結束
        elif ele.text.startswith('Description:'):
            description_li.append(ele.text[12:].strip())
#         將這個event id 的所有事件儲存至all_data
    all_data[str(i)]=[type_li, source_li, description_li]
#     每過1000個回合儲存一次
    if i % 1000 == 0:
        save_file(i)

100%|██████████████████████████████████████████| 11/11 [00:19<00:00,  1.60s/it]


In [9]:
def save_file(num):
    import codecs
    import pickle
    global all_data
# 若要更改儲存路徑，請更改下列path
# 儲存成pickle檔案形式，如此的話之後可以直接讀取，無須再parse文字檔
    with open('C:/Users/CHAN CHI/Event_dictionary/all_event_data_until'+str(num),'wb') as fout:
        pickle.dump(all_data,fout)
# 儲存成文字檔案形式，方便人眼觀察
    with codecs.open('C:/Users/CHAN CHI/Event_dictionary/all_event_data_until'+str(num)+'.txt','w','utf-8') as fout:
        fout.write(str(all_data))

In [16]:
import pickle
from collections import Counter
import operator
import codecs
# 讀取pickle檔，此路徑需與save_file function的pickle檔路徑一致
with open('C:/Users/CHAN CHI/Event_dictionary/all_event_data_until30000','rb') as fin:
    data=pickle.load(fin)
Sources=[]
text=''
# 計算有幾個dinstinct source 及總共有幾個events
# for k, v in data.items():
#     for i in v[1]:
#         Sources.append(i)
# S=Counter(Sources)
# total_events_num=0
# print(len(S))
# for k, v in sorted(S.items(), key=lambda x: x[1], reverse=True):
# #     print(k,v)
#     total_events_num+=v
# print(total_events_num)

# 將抓取下來的檔案寫成EXCEL檔，檔案請用sublime 3開啟，用Windows的notepad好像因為換行符號不一致的關係，會沒有換行
# 用sublime 3開啟之後複製到excel檔，在剖析資料時以tab鍵區隔欄位資料，即完成
for k, v in data.items():
    for i in range(len(v[0])):
        text+=str(k)+'\t'+v[0][i]+'\t'+v[1][i]+'\t'+v[2][i].replace('\n',' ')+'\n'
#         print(v[2][i])
with codecs.open('C:/Users/CHAN CHI/Event_dictionary/final data'+'.txt','w','utf-8') as fout:
    fout.write(text)